In [10]:
import glob

In [11]:
import matplotlib.pyplot as plt

In [12]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

In [13]:
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

In [14]:
import os

In [15]:
from pathlib import Path

In [16]:
Path

pathlib.Path

In [17]:
datapath=Path('./Data/dogs_vs_cats/train')

In [18]:
def get_num_files(path):
    if not os.path.exists(path):
        return 0
    else:
        return sum([len(files) for r,d,files in os.walk(path)])

In [19]:
def get_num_subfolders(path):
    if not os.path.exists(path):
        return 0
    else:
        return sum([len(d) for r,d,files in os.walk(path)])

In [20]:
def create_img_generator():
    return ImageDataGenerator(preprocessing_function=preprocess_input,
                      shear_range=0.2,rotation_range=0.2,
                      width_shift_range=.2,
                      height_shift_range=.2,
                      horizontal_flip=True,
                      zoom_range=0.2)

In [21]:
img_width,img_height=299,299
trainning_epochs=2
Batch_size=32
number_FC_neurons=1024

In [22]:
train_dir='./Data/dogs_vs_cats/train'
val_dir='./Data/dogs_vs_cats/validate'

In [23]:
num_train_files=get_num_files(train_dir)
num_classes=get_num_subfolders(train_dir)
num_validation_files=get_num_files(val_dir)

In [24]:
train_img_genrator=create_img_generator()
test_img_generator=create_img_generator()

In [25]:
num_epoch=trainning_epochs
batch_size=Batch_size

In [26]:
train_generator=train_img_genrator.flow_from_directory(
                    train_dir,
                    target_size=(img_height,img_width),
                    batch_size=batch_size,
                        seed=42)

Found 2000 images belonging to 2 classes.


In [27]:
validatioin_generator=train_img_genrator.flow_from_directory(
                    val_dir,
                    target_size=(img_height,img_width),
                    batch_size=batch_size,
                        seed=42)

Found 838 images belonging to 2 classes.


In [28]:
InceptionV3_base_model=InceptionV3(weights='imagenet',include_top=False)

In [29]:
x=InceptionV3_base_model.output

In [30]:
x=GlobalAveragePooling2D()(x)
x=Dense(number_FC_neurons,activation='relu')(x)
predictions=Dense(num_classes,activation='softmax')(x)

In [31]:
model=Model(input=InceptionV3_base_model.input,outputs=predictions)
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

C:\Users\91911\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [32]:
for layer in InceptionV3_base_model.layers:
    layer.trainable=False

In [33]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [34]:
history_transfer_learning=model.fit_generator(train_generator,
                                             epochs=num_epoch,
                                             steps_per_epoch=num_train_files//batch_size,
                                             validation_data=validatioin_generator,
                                             validation_steps=num_validation_files//batch_size,
                                             class_weight='auto')
model.save('InceptionV3-transfer-learning.model')

Epoch 1/2
62/62 [==============================] - 127s 2s/step - loss: 0.2328 - accuracy: 0.9096 - val_loss: 0.0152 - val_accuracy: 0.9267
Epoch 2/2
62/62 [==============================] - 115s 2s/step - loss: 0.1346 - accuracy: 0.9527 - val_loss: 0.5762 - val_accuracy: 0.8412


In [35]:
layers_to_freeze=172

In [37]:
for layer in model.layers[:layers_to_freeze]:
    layer.trainable=False
for layer in model.layers[layers_to_freeze:]:
    layer.trainable=True
    
model.compile(optimizer=SGD(lr=0.0001,momentum=0.9),loss='categorical_crossentropy',metrics=['accuracy'])

istory_transfer_learning_fine_tuning=model.fit_generator(train_generator,
                                             epochs=num_epoch,
                                             steps_per_epoch=num_train_files//batch_size,
                                             validation_data=validatioin_generator,
                                             validation_steps=num_validation_files//batch_size,
                                             class_weight='auto')
model.save('InceptionV3-transfer-learning_fine_tuning.model')

Epoch 1/2


ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,147,147] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node SquaredDifference_2 (defined at C:\Users\91911\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_71606]

Function call stack:
keras_scratch_graph
